In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import os
import random
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVR, SVC

In [3]:
a = "label_train_test.pkl"
with open(a, 'rb') as f:
    label, train, test = pickle.load(f)
print(len(train), len(test))

2252 551


In [41]:
data_path = './gnet2'
all_data = os.listdir(data_path)
print('# files', len(all_data))
data = []
all_labels = set()
for file_ in all_data:
    basename = file_
    name = os.path.splitext(basename)[0]
    data.append(os.path.join(data_path, basename))
    label_ = label[name]
    ls = label_.split('+')
    for l in ls:
        all_labels.add(l)
    # print(name, label_)
print(len(all_labels), all_labels)

# files 2706
14 {'HSIL', 'HSV', 'TRI', 'VIRUS', 'EC', 'LSIL', 'CC', 'SCC', 'ASCH', 'AGC', 'ACTINO', 'NILM', 'ASCUS', 'FUNGI'}


In [42]:
df = pd.read_csv(data[5])
# patches = float(df.patches[df.patches.notnull()])
# print(patches)
df.head(10)

,mpp,area,patches,patch_i,patch_label,patch_probability,detect_label,detect_probability,classify_label,classify_probability,x,y,w,h
0,NaN,NaN,NaN,69,HSIL-SCC_G,0.464286,TRI,0.274466,TRI,1.000000,5686,20344,24,22
1,NaN,NaN,NaN,667,NORMAL,0.455652,TRI,0.611174,TRI,1.000000,18504,16168,22,22
2,NaN,NaN,NaN,1371,NORMAL,0.919858,TRI,0.181044,TRI,0.999999,31380,30986,32,40
3,NaN,NaN,NaN,652,TRI,0.989159,TRI,0.105297,TRI,0.999995,17590,48734,32,26
4,NaN,NaN,NaN,2336,SC,0.998384,SC,0.637666,SC,0.999993,47816,14838,38,36
5,NaN,NaN,NaN,1053,HSIL-SCC_G,0.999529,ACTINO,0.119227,ACTINO,0.999993,25328,30742,160,164
6,NaN,NaN,NaN,1040,NORMAL,0.962150,HSIL-SCC_G,0.284843,HSIL_S,0.999986,25548,19042,74,64
7,NaN,NaN,NaN,911,HSIL-SCC_G,0.992527,HSIL-SCC_G,0.275052,HSIL_B,0.999973,22694,46670,150,226
8,NaN,NaN,NaN,1855,HSIL-SCC_G,0.990912,HSIL-SCC_G,0.863593,HSIL_B,0.999967,38498,37574,290,214
9,NaN,NaN,NaN,1574,NORMAL,0.782509,HSIL-SCC_G,0.451382,HSIL_S,0.999967,34336,22272,64,54


### features

In [43]:
tolerate = {"AGC":{"AGC_A", "AGC_B"}, 
            "LSIL":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "ASCUS":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "HSIL-SCC_G":{"HSIL_B", "HSIL_M", "HSIL_S", "SCC_G"}, 
            "SCC_R":{"SCC_R"}, 
            "EC":{"EC"}, 
            "CC":{"CC"}, 
            "VIRUS":{"VIRUS"}, 
            "FUNGI":{"FUNGI"}, 
            "ACTINO":{"ACTINO"}, 
            "TRI":{"TRI"}, 
            "PH":{"PH"}, 
            "SC":{"SC", "RC", "MC", "GEC"}}

dtct_p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
clas_p = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99, 0.999]
header = ["{}_{:.2f}_{:.3f}".format(key, dp, cp) for key in tolerate for dp in dtct_p for cp in clas_p]
header.sort()
header_map = {key:i for i,key in enumerate(header)}

all_labels = {'ACTINO':0, 'AGC':1, 'ASCH':2, 'ASCUS':3, 'CC':4, 'EC':5, 'FUNGI':6, 'HSIL':7, 'HSV':8, 'LSIL':9, 'NILM':10, 'SCC':11, 'TRI':12, 'VIRUS':8}
bin_labels = {}
for l in all_labels:
    if l == 'NILM':
        bin_labels[l] = 0
    else:
        bin_labels[l] = 1
print(all_labels)
print(bin_labels)

{'HSIL': 7, 'HSV': 8, 'TRI': 12, 'VIRUS': 8, 'EC': 5, 'CC': 4, 'SCC': 11, 'ASCH': 2, 'AGC': 1, 'ACTINO': 0, 'NILM': 10, 'LSIL': 9, 'ASCUS': 3, 'FUNGI': 6}
{'HSIL': 1, 'HSV': 1, 'TRI': 1, 'VIRUS': 1, 'EC': 1, 'CC': 1, 'SCC': 1, 'ASCH': 1, 'AGC': 1, 'ACTINO': 1, 'NILM': 0, 'LSIL': 1, 'ASCUS': 1, 'FUNGI': 1}


In [44]:
def extract_oldfashion(f):
    df = pd.read_csv(f)
    features = [0 for i in range(len(header))]
    # check if is empty
    if df.yolo_cell_class.isnull().values.any():  
        return features
    # cross levelup features
    for i,row in df.iterrows():
        for dp in dtct_p:
            for cp in clas_p:
                if row['xcp_cell_class'] in tolerate[row['yolo_cell_class']]:
                    if row['yolo_cell_class_det'] > dp and row['xcp_cell_class_det'] > cp:
                        key = "{}_{:.2f}_{:.3f}".format(row['yolo_cell_class'], dp, cp)
                        features[header_map[key]] += 1
    return features

def extract(f):
    df = pd.read_csv(f)
    features = [0 for i in range(len(header))]
#     areas = [0.0 for i in range(len(header))]
    # check if is empty
    if df.detect_label.isnull().values.any():  
        return features
    # cross levelup features
    for i,row in df.iterrows():
        for dp in dtct_p:
            for cp in clas_p:
                if row['classify_label'] in tolerate[row['detect_label']]:
                    if row['detect_probability'] > dp and row['classify_probability'] > cp:
                        key = "{}_{:.2f}_{:.3f}".format(row['detect_label'], dp, cp)
                        features[header_map[key]] += 1
#                         areas[header_map[key]] += row['w'] * row['h']
#     # average areas
#     for i in range(len(header)):
#         areas[i] /= features[i] if features[i] != 0 else 1.0
    
#     # patches balanced numbers
#     try:
#         patches = float(df.patches[df.patches.notnull()])
#         features_pb = [f*2000/patches for f in features]
#         features += features_pb
#     except:
#         print(f)
#         features *= 2

#     features += areas
    return features

def collect(data):
    X = []
    ya = []  # all labels
    yb = []  # binary labels
    names = []
    for f in data:
        features = extract(f)
        basename = os.path.splitext(os.path.basename(f))[0]
        ls = label[basename].split('+')
        if sum(features) == 0:
            continue
        for l in ls:
            a = all_labels[l]
            b = bin_labels[l]
            X.append(features)
            ya.append(a)
            yb.append(b)
            names.append(f)
    return X, ya, yb, names

def worker():
    files = data
    random.shuffle(files)
    random.shuffle(files)
    print("# files:", len(files))

    X, ya, yb, names = [], [], [], []
    
    executor = ProcessPoolExecutor(max_workers=36)
    tasks = []

    batch_size = 50
    for i in range(0, len(files), batch_size):
        batch = files[i : i+batch_size]
#         collect(batch)
        tasks.append(executor.submit(collect, batch))

    job_count = len(tasks)
    for future in as_completed(tasks):
        X_, ya_, yb_, names_ = future.result()  # get the returning result from calling fuction
        X += X_
        ya += ya_
        yb += yb_
        names += names_
        job_count -= 1
        if job_count % 8 == 0: 
            print("One Job Done, Remaining Job Count: %s" % (job_count))

    X = np.asarray(X)
    ya = np.asarray(ya)
    yb = np.asarray(yb)
    print(X.shape, ya.shape, yb.shape)
    
    return X, ya, yb, names

X, ya, yb, names = worker()

# files: 2706
One Job Done, Remaining Job Count: 48
One Job Done, Remaining Job Count: 40
One Job Done, Remaining Job Count: 32
One Job Done, Remaining Job Count: 24
One Job Done, Remaining Job Count: 16
One Job Done, Remaining Job Count: 8
One Job Done, Remaining Job Count: 0
(2785, 1170) (2785,) (2785,)


### classification

In [45]:
class RFESVM:
    def __init__(self):
        self.estimator = SVR(kernel="linear")
        self.selector = None
        
    def select(self, X, y, num_feature):
        self.selector = RFE(self.estimator, num_feature, step=1)
        self.selector = self.selector.fit(X, y)
        selected_feature_indices = self.selector.support_ # ndarray of True/False
        return selected_feature_indices

def split(X, y, mode, test_size, seed):
    random.seed(seed)
    N = 2 if mode == "bin" else 13
    idx = {i:[] for i in range(N)}
    for i,c in enumerate(y):
        idx[c].append(i)
    idx_t, idx_v = [], []
    for c,indices in idx.items():
        n = len(indices)
        idx_t += indices[:-int(n*test_size)]
        idx_v += indices[-int(n*test_size):]
    X_train = X[idx_t]
    X_valid = X[idx_v]
    y_train = y[idx_t]
    y_valid = y[idx_v]
    return X_train, X_valid, y_train, y_valid

def rfe(X_train, X_valid, y_train, y_valid, num_features):
    rfe_svm = RFESVM()
    selected_feature_indices = rfe_svm.select(X_train, y_train, num_features)
    X_train = X_train[:, selected_feature_indices] # Select elements of numpy array via boolean mask array
    X_valid = X_valid[:, selected_feature_indices]
    return X_train, X_valid, y_train, y_valid

def evaluate(y_valid, y_pred, mode):
    if mode == "bin":
        cur_labels = {0:'NILM', 1:'ABN'}
    else:
        cur_labels = {value:key for key,value in all_labels.items()}
    
    results = {i:[0,0,0] for i in range(len(cur_labels))}  # tp, fn, fp
    for t,p in zip(y_valid, y_pred):
        if t == p:
            results[t][0] += 1
        else:
            results[t][1] += 1
            results[p][2] += 1
    for i in range(len(results)):
        labeli = cur_labels[i]
        recall = results[i][0] / (results[i][0] + results[i][1]) if results[i][0] + results[i][1] != 0 else 0.0
        precision = results[i][0] / (results[i][0] + results[i][2]) if results[i][0] + results[i][2] != 0 else 0.0
        print(labeli, results[i][0] + results[i][1], ' recall = {:.4f}'.format(recall), 'precision = {:.4f}'.format(precision))
        
    
def classify(mode="bin"):  # mode = "bin" or "all"
    seed = 2018
    test_size = 0.2
#     num_features = 1170

    y = yb if mode == "bin" else ya
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=seed)
    X_train, X_valid, y_train, y_valid = split(X, y, mode, test_size, seed)
    # X_train, X_valid, y_train, y_valid = rfe(X_train, X_valid, y_train, y_valid, num_features)

    model = XGBClassifier(max_depth=15, 
                          n_jobs=24, 
                          subsample=0.8, 
                          colsample_bylevel=1,
                          colsample_bytree=0.6, 
                          scale_pos_weight=1, 
                          n_estimators=500, 
                          min_child_weight=1, 
                          learning_rate=0.1, 
                          gamma=0,
                          random_state=seed)
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    if mode == "bin":
        eval_metric = ["auc", "error"]
    else:
        eval_metric = ["merror"]
    model.fit(X_train, y_train, 
              early_stopping_rounds=50, 
              eval_metric=eval_metric, 
              eval_set=eval_set, 
              verbose=False)

    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("accuracy: {:.4f}".format(accuracy))
    evaluate(y_valid, y_pred, mode)
    
    return model

print('binary classification')
bin_model = classify("bin")
print()
print('multilabel classification')
all_model = classify("all")

binary classification
accuracy: 0.8723
NILM 184  recall = 0.8043 precision = 0.8087
ABN 372  recall = 0.9059 precision = 0.9035

multilabel classification
accuracy: 0.7120
ACTINO 8  recall = 0.8750 precision = 0.8750
AGC 6  recall = 0.3333 precision = 0.5000
ASCH 23  recall = 0.0435 precision = 0.1111
ASCUS 82  recall = 0.4878 precision = 0.5970
CC 35  recall = 0.6571 precision = 0.7931
EC 7  recall = 0.4286 precision = 0.4286
FUNGI 60  recall = 0.7333 precision = 0.7719
HSIL 47  recall = 0.7234 precision = 0.7083
VIRUS 2  recall = 1.0000 precision = 1.0000
LSIL 69  recall = 0.7681 precision = 0.7571
NILM 184  recall = 0.8859 precision = 0.7376
SCC 13  recall = 0.9231 precision = 0.6316
TRI 16  recall = 0.5625 precision = 0.8182


In [46]:
bin_imap = {0:'NILM', 1:'ABN'}
all_imap = {0: 'ACTINO', 1: 'AGC', 2: 'ASCH', 3: 'ASCUS', 4: 'CC', 5: 'EC', 6: 'CANDIDA', 7: 'HSIL', 8: 'HSV', 9: 'LSIL', 10: 'NILM', 11: 'SCC', 12: 'TRI'}

with open("gnet2models.pkl", 'wb') as f:
    pickle.dump(bin_model, f)
    pickle.dump(all_model, f)
    pickle.dump(bin_imap, f)
    pickle.dump(all_imap, f)


# with open("gnet2models.pkl", 'rb') as f:
#     bin_model = pickle.load(f)
#     all_model = pickle.load(f)
#     bin_imap = pickle.load(f)
#     all_imap = pickle.load(f)

#### release

In [47]:
class Diagnoser:
    def __init__(self, pkl_file):
        with open(pkl_file, 'rb') as f:
            self.bin_model = pickle.load(f)
            self.all_model = pickle.load(f)
            self.bin_imap = pickle.load(f)
            self.all_imap = pickle.load(f)
    
    def extract_old(self, csv_file):
        df = pd.read_csv(csv_file)
        features = [0 for i in range(len(header))]
        # check if is empty
        if df.yolo_cell_class.isnull().values.any():  
            return features
        # cross levelup features
        for i,row in df.iterrows():
            for dp in dtct_p:
                for cp in clas_p:
                    if row['xcp_cell_class'] in tolerate[row['yolo_cell_class']]:
                        if row['yolo_cell_class_det'] > dp and row['xcp_cell_class_det'] > cp:
                            key = "{}_{:.2f}_{:.3f}".format(row['yolo_cell_class'], dp, cp)
                            features[header_map[key]] += 1
        features = np.array([features])
        return features
    
    def extract_new(self, csv_file):
        df = pd.read_csv(csv_file)
        features = [0 for i in range(len(header))]
        # check if is empty
        if df.detect_label.isnull().values.any():  
            return features
        # cross levelup features
        for i,row in df.iterrows():
            for dp in dtct_p:
                for cp in clas_p:
                    if row['classify_label'] in tolerate[row['detect_label']]:
                        if row['detect_probability'] > dp and row['classify_probability'] > cp:
                            key = "{}_{:.2f}_{:.3f}".format(row['detect_label'], dp, cp)
                            features[header_map[key]] += 1
        features = np.array([features])
        return features
    
    def bin_predict(self, csv_file):
        f = self.extract_new(csv_file)
        p = self.bin_model.predict(f)[0]
        l = self.bin_imap[p]
        return l
        
    def all_predict(self, csv_file):
        f = self.extract_new(csv_file)
        p = self.all_model.predict(f)[0]
        l = self.all_imap[p]
        return l

    def bin_and_all_predict(self, csv_file):
        f = self.extract_new(csv_file)
        pb = self.bin_model.predict(f)[0]
        lb = self.bin_imap[pb]
        pa = self.all_model.predict(f)[0]
        la = self.all_imap[pa]
        return lb, la

In [49]:
pkl_file = "gnet2models.pkl"
d = Diagnoser(pkl_file)

csv_file = data[111]
l = d.all_predict(csv_file)
print(l)

lb, la = d.bin_and_all_predict(csv_file)
print(lb, la)

ASCUS
ABN ASCUS


### cross validation

In [ ]:
def classify(mode="bin"):  # mode = "bin" or "all"
    seed = 2018
    
    y = yb if mode == "bin" else ya
    num_folds = 5
    X_train_folds = np.array_split(X, num_folds)
    y_train_folds = np.array_split(y, num_folds)
    
    best = [0.0, None] # accuracy, i
    for i in range(0, num_folds):
        X_train = np.concatenate(X_train_folds[:i] + X_train_folds[i+1:])
        y_train = np.concatenate(y_train_folds[:i] + y_train_folds[i+1:])
        X_valid = X_train_folds[i]
        y_valid = y_train_folds[i]

        model = XGBClassifier(max_depth=15, 
                              n_jobs=24, 
                              subsample=0.8, 
                              colsample_bylevel=1,
                              colsample_bytree=0.6, 
                              scale_pos_weight=1, 
                              n_estimators=500, 
                              min_child_weight=1, 
                              learning_rate=0.1, 
                              gamma=0,
                              random_state=seed)
        eval_set = [(X_train, y_train), (X_valid, y_valid)]
        if mode == "bin":
            eval_metric = ["auc", "error"]
        else:
            eval_metric = ["merror"]
        model.fit(X_train, y_train, 
                  early_stopping_rounds=50, 
                  eval_metric=eval_metric, 
                  eval_set=eval_set, 
                  verbose=False)

        y_pred = model.predict(X_valid)
        accuracy = accuracy_score(y_valid, y_pred)
        print("accuracy: {:.4f}".format(accuracy))
        
        if accuracy > best[0]:
            best = [accuracy, i]
            
    print(best)

In [ ]:
print('binary classification')
classify("bin")
print()
print('multilabel classification')
classify("all")